# Librerías

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from xgboost import XGBRegressor

# Datos de estaciones desde el 2000

In [2]:
df = pd.read_csv("../Datos/RMCAB/Clean/Sec_Cruzada_RMCAB.csv")
df["Fecha_Hora"] = pd.to_datetime(df["Fecha_Hora"])

estaciones = df["Estación"].unique().tolist()
info_estaciones = pd.DataFrame(columns=["Estación", "Min_Fecha_Hora"])

# Iterate over each station and find the minimum date and time
for estacion in estaciones:
    min_date = df[df["Estación"] == estacion]["Fecha_Hora"].min()
    new_row = pd.DataFrame({"Estación": [estacion], "Min_Fecha_Hora": [min_date]})
    info_estaciones = pd.concat([info_estaciones, new_row], ignore_index=True)

estaciones_filtradas = info_estaciones.head(7)["Estación"].tolist()

df = df[df["Estación"].isin(estaciones_filtradas)]

# Imputación para cada variable por estación

In [3]:
estaciones_filtradas

['Carvajal - Sevillana',
 'Guaymaral',
 'MinAmbiente',
 'Puente Aranda',
 'Suba',
 'Usaquen',
 'Las Ferias']

In [4]:
nombre_estacion = "Guaymaral"

e = df[df["Estación"] == nombre_estacion]
del df
#e = pd.read_csv(f"../Datos/RMCAB/Clean/Datos_sin_faltantes/{nombre_estacion}_imputado.csv")
e = e.set_index("Fecha_Hora")
var_to_imput = e.columns[1:].tolist()
e.isna().sum()

try:
    e = pd.read_csv(f"../Datos/RMCAB/Clean/Datos_sin_faltantes/{nombre_estacion}_imputado.csv")
except:
    e.to_csv(f"../Datos/RMCAB/Clean/Datos_sin_faltantes/{nombre_estacion}_imputado.csv")
    print("No se encontró el archivo, se creó uno nuevo")
e = e.set_index("Fecha_Hora")
e.isna().sum()

Estación                0
BBP                204769
BC ug/m3           204769
CO                 177160
Dir Viento         204769
Dir Viento 10M          0
HR                      0
HR_2m              204769
Humedad Inter      204769
NO                  95016
NO2                 95053
NOX                  7068
OZONO               64576
OZONO Envea        204769
PM10                 7099
PM2.5              109894
Precipitacion           0
Presion Baro            0
Rad Solar               0
SO2                201165
SO2 Envea          204769
Temp_4m            204769
Tempe Inter        204769
Temperatura         62306
Temperatura 20M     62306
Temperatura 8M      62306
UV-BC              204769
Vel Viento         204769
Vel Viento 10M          0
dtype: int64

In [5]:
# Iterar sobre cada variable a imputar
for var in var_to_imput:
    disp = 1 - (e[var].isna().sum() / len(e[var]))  # Calcular la disponibilidad de datos
    #print(var, disp, len(e[var]))
    # Imputar solo si la disponibilidad de datos es suficiente
    if disp >= 0.6: 

        e = pd.read_csv(f"../Datos/RMCAB/Clean/Datos_sin_faltantes/{nombre_estacion}_imputado.csv")   
        e = e.set_index("Fecha_Hora")
        min_date = e[var].dropna().index.min()
        df_to_replace = e[var].loc[e[var].index >= min_date]
        df_to_replace = pd.DataFrame(df_to_replace)
        #print(df_to_replace)
        n_lags = 50
        correlations = [e[var].autocorr(lag) for lag in range(1, n_lags+1)]
        correlation_df = pd.DataFrame({'Lag': range(1, n_lags+1), 'Correlation': correlations})
        top_lags = correlation_df.sort_values(by='Correlation', ascending=False)
        best_lags = top_lags.head(5)["Lag"].tolist()
        
        imputed_count = 0  # Contador de valores imputados

        # Imputar los valores nulos uno a uno
        while df_to_replace[var].isna().sum() > 0:
            next_nan_value_date = df_to_replace[df_to_replace.isna().any(axis=1)].index[0]
            
            model_df = df_to_replace.loc[df_to_replace.index <= next_nan_value_date]
            model_df = pd.DataFrame(model_df)

            # Crear variables de lag
            for lag in best_lags:
                model_df[f'lag_{lag}'] = model_df[var].shift(lag)

            next_nan_value = model_df.tail(1)
            model_df = model_df.dropna()

            # Preparar los datos de entrenamiento
            X = model_df.drop(columns=[var])
            y = model_df[var]

            # Reentrenar el modelo cada 100 imputaciones
            if imputed_count % 100 == 0 or imputed_count == 0:
                model = XGBRegressor()
                model.fit(X, y)
                #print(f"Modelo reentrenado en la imputación {imputed_count}")

            # Predecir el siguiente valor nulo y actualizar el DataFrame
            y_pred = model.predict(next_nan_value.drop(columns=var))
            e[var].loc[next_nan_value_date] = y_pred.round(0)
            df_to_replace[var].loc[next_nan_value_date] = y_pred.round(0)

            imputed_count += 1  # Actualizar el contador
            #print(f"Imputación realizada en {next_nan_value_date}: {y_pred.round(0)}")
    if disp < 0.6:
        print(f"La disponibilidad de datos para {var} es insuficiente para imputar")
    e.to_csv(f"../Datos/RMCAB/Clean/Datos_sin_faltantes/{nombre_estacion}_imputado.csv")

La disponibilidad de datos para BBP es insuficiente para imputar
La disponibilidad de datos para BC ug/m3 es insuficiente para imputar
La disponibilidad de datos para CO es insuficiente para imputar
La disponibilidad de datos para Dir Viento es insuficiente para imputar
La disponibilidad de datos para HR_2m es insuficiente para imputar
La disponibilidad de datos para Humedad Inter es insuficiente para imputar
La disponibilidad de datos para NO es insuficiente para imputar
La disponibilidad de datos para NO2 es insuficiente para imputar
La disponibilidad de datos para OZONO Envea es insuficiente para imputar
La disponibilidad de datos para PM2.5 es insuficiente para imputar
La disponibilidad de datos para SO2 es insuficiente para imputar
La disponibilidad de datos para SO2 Envea es insuficiente para imputar
La disponibilidad de datos para Temp_4m es insuficiente para imputar
La disponibilidad de datos para Tempe Inter es insuficiente para imputar
La disponibilidad de datos para UV-BC es

In [6]:
e.isna().sum()

Estación                0
BBP                204769
BC ug/m3           204769
CO                 177160
Dir Viento         204769
Dir Viento 10M          0
HR                      0
HR_2m              204769
Humedad Inter      204769
NO                  95016
NO2                 95053
NOX                  7068
OZONO               64576
OZONO Envea        204769
PM10                 7099
PM2.5              109894
Precipitacion           0
Presion Baro            0
Rad Solar               0
SO2                201165
SO2 Envea          204769
Temp_4m            204769
Tempe Inter        204769
Temperatura         62306
Temperatura 20M     62306
Temperatura 8M      62306
UV-BC              204769
Vel Viento         204769
Vel Viento 10M          0
dtype: int64